Conexión a la base de datos

In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Si Cassandra está protegido por contraseña, agrega la autenticación
# auth_provider = PlainTextAuthProvider(username='your_username', password='your_password')

cluster = Cluster(['127.0.0.1'], port=9042) 
session = cluster.connect()

session.set_keyspace('system')

print("Conexión exitosa a Cassandra")


Conexión exitosa a Cassandra


Inserción de datos en cassandra

In [2]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
from datetime import datetime

# Configura la dirección IP del nodo Cassandra
CASSANDRA_HOST = '127.0.0.1'  
CASSANDRA_PORT = 9042  

# Nombre del Keyspace y la tabla
KEYSPACE = 'mykeyspace'
TABLE = 'mytable'

# Cargar los datos desde un archivo CSV (ajusta la ruta del archivo)
CSV_FILE = 'dataset_extracto.csv' 

def connect_to_cassandra():
    """Conectar a Cassandra y retornar la sesión."""
    cluster = Cluster([CASSANDRA_HOST], port=CASSANDRA_PORT)
    session = cluster.connect()
    return session

def create_keyspace_and_table(session):
    """Crear un Keyspace y una tabla en Cassandra."""
    # Crear Keyspace si no existe
    session.execute(f"""
        CREATE KEYSPACE IF NOT EXISTS {KEYSPACE} 
        WITH replication = {{'class': 'SimpleStrategy', 'replication_factor': 1}};
    """)

    # Usar el Keyspace creado
    session.set_keyspace(KEYSPACE)

    # Crear la tabla
    session.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE} (
            id INT PRIMARY KEY, 
            nombre TEXT,
            edad INT,
            email TEXT,
            fecha_registro TIMESTAMP
        );
    """)

def insert_data_from_csv(session):
    """Leer datos de CSV y cargarlos en Cassandra."""
    # Leer el archivo CSV con pandas
    df = pd.read_csv(CSV_FILE)

    # Convertir 'fecha_registro' al formato de timestamp
    df['fecha_registro'] = pd.to_datetime(df['fecha_registro'], errors='coerce')

    # Crear la sentencia de inserción
    insert_stmt = session.prepare(f"""
        INSERT INTO {TABLE} (id, nombre, edad, email, fecha_registro) 
        VALUES (?, ?, ?, ?, ?)
    """)

    # Insertar cada fila del CSV en la tabla
    for index, row in df.iterrows():
        # Convertir el ID a int
        id_value = int(row['id'])  

        # Ejecutar la inserción de datos
        session.execute(insert_stmt, (id_value, row['nombre'], row['edad'], row['email'], row['fecha_registro']))

def main():
    # Conectar a Cassandra
    session = connect_to_cassandra()

    # Crear el Keyspace y la tabla (si no existen)
    create_keyspace_and_table(session)

    # Insertar datos desde el CSV
    insert_data_from_csv(session)

    print("Datos cargados exitosamente desde el CSV a Cassandra.")

if __name__ == "__main__":
    main()


Datos cargados exitosamente desde el CSV a Cassandra.


Consultas en Cassandra

In [4]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import time

def conectar_cassandra():
    try:
        # Conectar a Cassandra
        cluster = Cluster(['127.0.0.1']) 
        session = cluster.connect('mykeyspace')  

        # Medir el tiempo de inicio
        start_time = time.time()

        # Ejecutar la consulta
        query = "select * from mykeyspace.mytable ;" 
        stmt = SimpleStatement(query)
        resultados = session.execute(stmt)

        # Medir el tiempo final
        end_time = time.time()

        # Calcular el tiempo de ejecución
        tiempo_ejecucion = end_time - start_time
        print(f"La consulta tardó {tiempo_ejecucion:.4f} segundos en ejecutarse.")

        # Imprimir los resultados
        for registro in resultados:
            print(registro)

    except Exception as e:
        print("Error al conectar o ejecutar la consulta:", e)

    finally:
        # Cerrar la conexión
        if session:
            session.shutdown()  # Cierra la sesión
            print("La conexión a Cassandra ha sido cerrada")

# Llamar a la función para conectar y ejecutar la consulta
conectar_cassandra()


La consulta tardó 0.0025 segundos en ejecutarse.
Row(id=9527, edad=49, email='oroberts@example.net', fecha_registro=datetime.datetime(2020, 4, 15, 0, 0), nombre='Joshua Riddle')
Row(id=3788, edad=86, email='emiller@example.org', fecha_registro=datetime.datetime(2024, 8, 9, 0, 0), nombre='Michelle Johnson')
Row(id=7670, edad=74, email='tracywilliams@example.org', fecha_registro=datetime.datetime(2021, 4, 17, 0, 0), nombre='Mitchell Walker')
Row(id=2668, edad=47, email='zacharybrewer@example.net', fecha_registro=datetime.datetime(2023, 5, 15, 0, 0), nombre='Jeffery Malone')
Row(id=9993, edad=69, email='smithcharles@example.org', fecha_registro=datetime.datetime(2022, 8, 2, 0, 0), nombre='Martin Stevens')
Row(id=7664, edad=82, email='brownlinda@example.com', fecha_registro=datetime.datetime(2024, 8, 2, 0, 0), nombre='Raymond Santos')
Row(id=8091, edad=30, email='steelejeffrey@example.com', fecha_registro=datetime.datetime(2020, 6, 8, 0, 0), nombre='Kelly Green')
Row(id=6751, edad=98, emai